# Generate the Planets Like in the Game

This creates latex figures that recreate each of the planets in Iridis Alpha.
See the other file for doing the same but generating them as images.

First we load the planet textures from the source code. Each character is stored from top left to bottom right.

In [1]:
import re
colors = {
    "00": "",
    "01": "\\BACKGROUNDONE",
    "10": "\\BACKGROUNDTWO",
    "11": "\\CHARCOLOR",
}
charsets_files = ["../iridisalpha/src/graphics/planet_textures.asm"]
charsets = {}
for charsets_file in charsets_files:
    input_file = open(charsets_file,'r')
    charset_data = []
    curr_charset = []
    for l in input_file.readlines():
        if "CHARACTER" in l or "planet" in l:
            if charset_data:
                curr_charset += [charset_data]
            charset_data = []
        if "planet" in l:
            if curr_charset:
                charsets[charset_name] = curr_charset
            charset_name = l.strip()
            curr_charset = []
            continue

        m = re.findall(r"[0-1]{8}",l)
        if not m:
            continue
        bits = m[0]
        for i in range(0,7,2):
            bitpair = bits[i:i+2]
            charset_data += [colors[bitpair]]
            charset_data += [colors[bitpair]]
    if charset_data:
        curr_charset += [charset_data]
    charsets[charset_name] = curr_charset


In [2]:
planet_colors = {
    'planet1Charset' : ['brown','lightblue','lightgreen'],
    'planet2Charset' : ['gray','lightred','brown'],
    'planet3Charset' : ['yellow','white', 'lightred'],
    'planet4Charset' : ['lightblue','yellow', 'darkgray'],
    'planet5Charset' : ['lightgreen','lightred','lightred'],
}

Get the structure data

In [19]:
raw_structure_data = """
mediumStructureData  .BYTE $65,$67,$69,$6B,$FF        
                     .BYTE $64,$66,$68,$6A,$FE        
largestStructureData .BYTE $41,$43,$51,$53,$41,$43,$FF
                     .BYTE $60,$60,$50,$52,$60,$60,$FF
                     .BYTE $49,$4B,$4D,$4F,$6D,$6F,$FF
                     .BYTE $48,$4A,$4C,$4E,$6C,$6E,$FE
nextLargestStructure .BYTE $59,$5B,$FF
                     .BYTE $58,$5A,$FF    
                     .BYTE $55,$57,$FF                
                     .BYTE $54,$56,$FE                
warpGateData         .BYTE $75,$77,$7D,$7F,$FF        
                     .BYTE $74,$76,$7C,$7E,$FF        
                     .BYTE $71,$73,$79,$7B,$FF        
                     .BYTE $70,$72,$78,$7A,$FE        
littleStructureData  .BYTE $45,$47,$FF
                     .BYTE $44,$46,$FE
"""
structure_data = {}
struct_chars = []
for l in raw_structure_data.split('\n'):
    if any(d in l for d in ["Data","Structure"]):
        if struct_chars:
            structure_data[struct_name] = struct_chars
        struct_name = l[:21].strip()
        struct_chars = [l[27:].strip().split(',')[:-1]]
        continue
    if "BYTE" in l:
        struct_chars += [l[27:].strip().split(',')[:-1]]
structure_data[struct_name] = struct_chars
structure_data.keys()

dict_keys(['mediumStructureData', 'largestStructureData', 'nextLargestStructure', 'warpGateData', 'littleStructureData'])

Generate the planet following the steps in GenPlan as closely as possible.

In [60]:
import copy
import random

SCENE_WIDTH = 256
HEIGHT = 4

# The template for each planet has the sea as its surface.
blank_scene = []
for y in range(0,HEIGHT):
    l = []
    for x in range(0,SCENE_WIDTH,2):
        l += ["$40","$42"] if y == 0 else ["$00","$00"]
    blank_scene += [l]

land_start = random.randint(0,128)
land_length = random.randint(0,128)
land_length += 32

# The left shore of the land
for i, v in enumerate(['$5C','$5E']):
    blank_scene[0][land_start + i] = v

# Fill the land
for i in range(land_start+2, land_start+land_length, 2):
    blank_scene[0][i] = '$41'
    blank_scene[0][i+1] = '$43'

# The right shore of the land
for i, v in enumerate(['$5D','$5F']):
    blank_scene[0][land_start + land_length + i] = v

# Add a random structure every 13 to 29 bytes.
random_struct_list = [s for s in structure_data.keys() if "warp" not in s]
struct_offset = random.randint(13,29)
while struct_offset < SCENE_WIDTH:
    random_struct_name = random_struct_list[random.randint(0,3)]
    random_struct = structure_data[random_struct_name]
    for i,v in enumerate(random_struct):
        for j,b in enumerate(v):
            blank_scene[i][struct_offset+j] = b
    struct_offset += random.randint(13,29)

# Add the warp gates at the beginning and end
for struct_offset in (4,241):
    random_struct = structure_data["warpGateData"]
    for i,v in enumerate(random_struct):
        for j,b in enumerate(v):
            blank_scene[i][struct_offset+j] = b
    
blank_scene = list(reversed(blank_scene))

Create the latex figure files in the `planets` subdirectory, with names like `charsetPlanet1_randomPlanet`.

In [61]:
BIT_WIDTH = 8

def fillCharacter(c, byte_x_offset=0, byte_y_offset=0, grid_height=0):
    output = ""
    bit_y_offset = ((grid_height - (byte_y_offset * BIT_WIDTH))) - 1
    for j,v in enumerate(c):
        if not v:
            continue
        x = (byte_x_offset * BIT_WIDTH) + ((j % BIT_WIDTH) if j >= 7 else j)
        y = (bit_y_offset - int(j/BIT_WIDTH)) if j >= 7 else bit_y_offset
        output += f'\t\\fill[{v}] ({x},{y}) rectangle ++ (1,1);\n'
    return output

def createStructure(planet_name, scene_name, scene):
    grid_width = BIT_WIDTH * len(scene[0])
    grid_height = BIT_WIDTH * len(scene)

    header = """
\\begin{figure}[H]
  {
    \\setlength{\\tabcolsep}{3.0pt}
    \\setlength\cmidrulewidth{\heavyrulewidth} % Make cmidrule = 
    \\begin{adjustbox}{width=15cm,center}
      \\begin{tikzpicture}
"""

    output = header + '\n'
    output += f"\\def\BACKGROUNDONE{{{planet_colors[planet_name][0]}}}\n"
    output += f"\\def\BACKGROUNDTWO{{{planet_colors[planet_name][1]}}}\n"
    output += f"\\def\CHARCOLOR{{{planet_colors[planet_name][2]}}}\n"

    grid = f"\\draw[step=1.0,gray,very thin] (0,0) grid ({grid_width},{grid_height});"
    #output += f"\t{grid}\n"
    
    charset = charsets[planet_name]
    for byte_y_offset,ln in enumerate(scene):
        for byte_x_offset,hex_ref in enumerate(ln):
            char_index = int(hex_ref[1:],16) - 0x40
            if char_index < 0:
                continue
            c = charset[char_index]
            output += fillCharacter(c, byte_x_offset, byte_y_offset, grid_height)

    footer = f"""
      \\end{{tikzpicture}}
    \\end{{adjustbox}}
  }}
\\end{{figure}}
"""
    output += footer
    return output

header = """
\\begin{figure}[H]                          
{                                          
  \\setlength{\\tabcolsep}{3.0pt}            
  \\setlength\cmidrulewidth{\lightrulewidth}
"""

for planet_name in planet_colors:
    CHUNK_SIZE = 64
    figure = header
    for os in range(0, len(blank_scene[0]), CHUNK_SIZE):
        scene = []
        for y,v in enumerate(blank_scene):
            scene += [v[os:os+CHUNK_SIZE]]
        output = createStructure(planet_name, "Random", scene)
        file_name = f"../src/planets/{planet_name}_Random{os}.tex"
        output_file = open(file_name,'w')
        output_file.write(output)
        output_file.close()
        figure += "\\begin{subfigure}{1.0\\textwidth}\n"
        figure += f"\\input{{{planet_name}_Random{os}}}\n"
        figure += "\\end{subfigure}\n"

    footer = f"""
    }}\\caption[]{{Randomly Generated Planet from {planet_name}}}
    \\end{{figure}}
    """
    figure += footer
    of_name = f"../src/planets/{planet_name}_randomly_generated.tex"
    with open(of_name,'w') as output_file:
        output_file.write(figure)
    print(f"\subfile{{planets/{planet_name}_randomly_generated}}")


\subfile{planets/planet1Charset_randomly_generated}
\subfile{planets/planet2Charset_randomly_generated}
\subfile{planets/planet3Charset_randomly_generated}
\subfile{planets/planet4Charset_randomly_generated}
\subfile{planets/planet5Charset_randomly_generated}
